In [6]:
# ReflectionAgent的核心逻辑：执行-反思-优化循环

In [7]:
# 初始执行提示词：首次尝试解决问题的提示词，简单直接
INITIAL_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。请根据以下要求，编写一个Python函数。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。

要求: {task}

请直接输出代码，不要包含任何额外的解释。
"""


In [8]:
# 反思提示词：指示模型扮演“代码评审员”，对上一轮生成的代码进行批判性分析，并提供具体的可操作的反馈
REFLECT_PROMPT_TEMPLATE = """
你是一位极其严格的代码评审专家和资深算法工程师，对代码的性能有极致的要求。
你的任务是审查以下Python代码，并专注于找出其在<strong>算法效率</strong>上的主要瓶颈。

# 原始任务:
{task}

# 待审查的代码:
```python
{code}
```

请分析该代码的时间复杂度，并思考是否存在一种<strong>算法上更优</strong>的解决方案来显著提升性能。
如果存在，请清晰地指出当前算法的不足，并提出具体的、可行的改进算法建议（例如，使用筛法替代试除法）。
如果代码在算法层面已经达到最优，才能回答“无需改进”。

请直接输出你的反馈，不要包含任何额外的解释。
"""


In [9]:
# 优化提示词：收到反馈后，这个提示词将引导模型根据反馈内容，对原有代码进行修正和优化

REFINE_PROMPT_TEMPLATE = """
你是一位资深的Python程序员。你正在根据一位代码评审专家的反馈来优化你的代码。

# 原始任务:
{task}

# 你上一轮尝试的代码:
{last_code_attempt}
评审员的反馈：
{feedback}

请根据评审员的反馈，生成一个优化后的新版本代码。
你的代码必须包含完整的函数签名、文档字符串，并遵循PEP 8编码规范。
请直接输出优化后的代码，不要包含任何额外的解释。
"""


In [10]:
# Reflection智能体封装与实现
from helloLLM import HelloAgentsLLM
from memory import Memory

class ReflectionAgent:
    """
    ReflectionAgent 的 Docstring
    能够完成[执行-反思-优化]循环的agent
    """
    def __init__(self, llm_client, max_iterations=3):
        self.llm_client = llm_client
        self.memory = Memory()
        self.max_iterations = max_iterations

    def run(self, task:str):
        print(f"\n--- 开始处理任务 ---\n任务：{task}")

        # 初始执行
        print("\n--- 正在进行初始尝试 ---")
        initial_prompt = INITIAL_PROMPT_TEMPLATE.format(task=task)
        initial_code = self._get_llm_response(initial_prompt)
        self.memory.add_record("execution", initial_code)

        # 迭代循环：反思与优化
        for i in range(self.max_iterations):
            print(f"\n--- 第{i+1}/{self.max_iterations} 轮迭代 ---")

            # 反思
            print("\n-> 正在进行反思...")
            last_code = self.memory.get_last_execution()
            reflect_prompt = REFLECT_PROMPT_TEMPLATE.format(task=task, code=last_code)
            feedback = self._get_llm_response(reflect_prompt)
            self.memory.add_record("reflection", feedback)

            # 检查是否需要停止
            if "无需改进" in feedback:
                print("\n反思认为代码已无需改进，任务完成")
                break

            # 优化
            print("\n-> 正在进行优化...")
            refine_prompt = REFINE_PROMPT_TEMPLATE.format(
                task=task,
                last_code_attempt=last_code,
                feedback=feedback
            )
            refined_code = self._get_llm_response(refine_prompt)
            self.memory.add_record("execution", refined_code)

        final_code = self.memory.get_last_execution()
        print(f"\n--- 任务完成 ---\n最终生成的代码：\n```python\n{final_code}\n```")
        return final_code

    def _get_llm_response(self, prompt: str) -> str:
        """
        _get_llm_response 的 Docstring
        一个辅助方法，用于调用LLM并获取完整的流式响应

        :param self: 说明
        :param prompt: 说明
        :type prompt: str
        :return: 说明
        :rtype: str
        """
        messages = [{"role": "user", "content": prompt}]
        response_text = self.llm_client.think(messages=messages) or ""
        return response_text

In [11]:
# 测试样例
task = "编写一个Python函数，找出1到n之间所有的素数 (prime numbers)"

# 初始化实例
myLLM = HelloAgentsLLM()
myAgent = ReflectionAgent(myLLM)

myAgent.run(task)


--- 开始处理任务 ---
任务：编写一个Python函数，找出1到n之间所有的素数 (prime numbers)

--- 正在进行初始尝试 ---
正在调用gemini-2.0-flash-free模型...
大语言模型响应成功：

模型原始输出：
```python
def find_primes(n: int) -> list[int]:
    """
    Finds all prime numbers between 1 and n (inclusive).

    Args:
        n: The upper limit of the range to search for prime numbers.

    Returns:
        A list of prime numbers found within the specified range.
        Returns an empty list if n is less than 2.
    """
    if n < 2:
        return []

    primes = []
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False

    for p in range(2, int(n**0.5) + 1):
        if is_prime[p]:
            for i in range(p*p, n + 1, p):
                is_prime[i] = False

    for p in range(2, n + 1):
        if is_prime[p]:
            primes.append(p)

    return primes
```
记忆已更新，新增一条'execution'记录

--- 第1/3 轮迭代 ---

-> 正在进行反思...
正在调用gemini-2.0-flash-free模型...
大语言模型响应成功：

模型原始输出：
时间复杂度：O(n log log n)。算法层面已最优，无需改进。

记忆已更新，新增一条'reflection'记录


'```python\ndef find_primes(n: int) -> list[int]:\n    """\n    Finds all prime numbers between 1 and n (inclusive).\n\n    Args:\n        n: The upper limit of the range to search for prime numbers.\n\n    Returns:\n        A list of prime numbers found within the specified range.\n        Returns an empty list if n is less than 2.\n    """\n    if n < 2:\n        return []\n\n    primes = []\n    is_prime = [True] * (n + 1)\n    is_prime[0] = is_prime[1] = False\n\n    for p in range(2, int(n**0.5) + 1):\n        if is_prime[p]:\n            for i in range(p*p, n + 1, p):\n                is_prime[i] = False\n\n    for p in range(2, n + 1):\n        if is_prime[p]:\n            primes.append(p)\n\n    return primes\n```'